In [1]:
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Popular Relevance").getOrCreate()

In [2]:
#load csv file
csvFile = "../retrotech/signals.csv"
csvDF = spark.read.format("csv").option("header", "true").load(csvFile)
csvDF.createOrReplaceTempView("signals")

In [3]:
signals_aggregation_query = """
select lower(q.target) as query, c.target as doc, count(c.target) as boost
  from signals c left join signals q on c.query_id = q.query_id
  where c.type = 'click' AND q.type = 'query'
  group by query, doc
  order by boost desc
"""
res=spark.sql(signals_aggregation_query)
res.coalesce(1).write.format("csv").option("header", "false").save("signals")